# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [3]:
# Set the geographical coordinates for Sydney, Australia
latitude = -33.8698439
longitude = 151.2082848

# Set the parameters for the type of place
categories = "accommodation.hotel"
#conditions = "vegetarian"
radius = 8000

# Set the parameters for the type of search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 20

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
print(response.url)

https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=20&filter=circle%3A151.2082848%2C-33.8698439%2C8000&bias=proximity%3A151.2082848%2C-33.8698439&apiKey=a5e6e9b4ac5c42e7862da503d2b20218


In [4]:
places_data= response.json()
pprint.pprint(places_data)

{'features': [{'geometry': {'coordinates': [151.2076595, -33.87093729966715],
                            'type': 'Point'},
               'properties': {'address_line1': 'QT Sydney',
                              'address_line2': '49 Market Street, Sydney NSW '
                                               '2000, Australia',
                              'categories': ['accommodation',
                                             'accommodation.hotel'],
                              'city': 'Sydney',
                              'contact': {'phone': '+61 2 8262 0062'},
                              'country': 'Australia',
                              'country_code': 'au',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database License',
                                             'raw': {'addr:housenumber': 49,
  

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,289.49,79,40,3.60,CA,1724868340
1,1,shingu,33.7333,135.9833,298.49,91,100,1.80,JP,1724868341
2,2,tura,25.5198,90.2201,297.77,89,26,1.39,IN,1724868342
3,3,avarua,-21.2078,-159.7750,297.18,78,75,6.69,CK,1724868343
4,4,ilulissat,69.2167,-51.1000,279.16,70,75,1.54,GL,1724868344


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', size='Humidity', colorbar=True, 
                                       cmap='coolwarm', title='City Humidity Map', 
                                       frame_width=800, frame_height=600)

# Display the map plot
map_plot


:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_city = pd.DataFrame(city_data_df.loc[(city_data_df['Cloudiness']==0),:])
# Drop any rows with null values
# YOUR CODE HERE
city_data_df.dropna(inplace=True)
# Display sample data
# YOUR CODE HERE
ideal_weather_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,petropavlovsk-kamchatsky,53.0452,158.6483,281.48,87,0,1.00,RU,1724868407
20,20,umea,63.8284,20.2597,288.11,94,0,2.06,SE,1724868411
25,25,fale old settlement,-9.3852,-171.2468,301.16,73,0,7.05,TK,1724868416
28,28,tolanaro,-25.0319,46.9987,294.58,76,0,7.72,MG,1724868420
63,63,wailua homesteads,22.0669,-159.3780,299.59,86,0,4.63,US,1724868460
...,...,...,...,...,...,...,...,...,...,...
545,545,yicheng,31.3705,119.8703,298.99,92,0,3.16,CN,1724869026
552,552,kerman,30.2832,57.0788,299.07,17,0,2.06,IR,1724868905
557,557,zaragoza,41.6561,-0.8773,306.40,40,0,6.17,ES,1724869042
558,558,ash shamiyah,31.9636,44.5990,312.20,17,0,3.09,IQ,1724869043


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
columns = ['City','Country','Lat','Lng','Humidity']
hotel_df = ideal_weather_city[columns].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
17,petropavlovsk-kamchatsky,RU,53.0452,158.6483,87,
20,umea,SE,63.8284,20.2597,94,
25,fale old settlement,TK,-9.3852,-171.2468,73,
28,tolanaro,MG,-25.0319,46.9987,76,
63,wailua homesteads,US,22.0669,-159.3780,86,
...,...,...,...,...,...,...
545,yicheng,CN,31.3705,119.8703,92,
552,kerman,IR,30.2832,57.0788,17,
557,zaragoza,ES,41.6561,-0.8773,40,
558,ash shamiyah,IQ,31.9636,44.5990,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
categories = "accommodation.hotel"  #for hotel
limit = 10  #get first hotel

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    try:
        latitude = row.Lat
        longitude = row.Lng 
        #print(latitude, longitude)
        # Add the current city's latitude and longitude to the params dictionary
        filters = f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
        bias = f"proximity:{longitude},{latitude}" # YOUR CODE HEREHERE
        
        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"
        # YOUR CODE HERE
        hotel_params = {
            "categories":categories,
            "filter":filters, 
            "limit":limit,
            "bias":bias, #will be created for each location in the loop
            "apiKey":geoapify_key
        }
        #print(hotel_params)
        # Make and API request using the params dictionary
        hotel_name = requests.get(base_url, params=hotel_params)
        #print(hotel_name.url)
        # Convert the API response to JSON format
        hotel_name = hotel_name.json() # YOUR CODE HERE
        #pprint.pprint(hotel_name)
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    except Exception as e:
        print(e+row)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df


Starting hotel search
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
umea - nearest hotel: Hotell Dragonen
fale old settlement - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
alto araguaia - nearest hotel: Hotel Araguaia
newman - nearest hotel: No hotel found
nur-sultan - nearest hotel: Тарлан
empalme - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
comodoro rivadavia - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
lihue - nearest hotel: Kauai Palms
lappeenranta - nearest hotel: Sokos Hotel Lappee
patterson - nearest hotel: Hampton Inn & Suites
fort st. john - nearest hotel: Fort St. John Motor Inn
arraial do cabo - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
puerto real - nearest hotel: Hostal Bahía de Cádiz
kill devil hills - nearest hotel: Mariner Days Inn & Suit

,City,Country,Lat,Lng,Humidity,Hotel Name
17,petropavlovsk-kamchatsky,RU,53.0452,158.6483,87,ООО Постоялый двор
20,umea,SE,63.8284,20.2597,94,Hotell Dragonen
25,fale old settlement,TK,-9.3852,-171.2468,73,No hotel found
28,tolanaro,MG,-25.0319,46.9987,76,Hôtel Mahavokey
63,wailua homesteads,US,22.0669,-159.3780,86,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...,...
545,yicheng,CN,31.3705,119.8703,92,Le Méridien Yixing
552,kerman,IR,30.2832,57.0788,17,حافظ
557,zaragoza,ES,41.6561,-0.8773,40,Hotel Pilar Plaza
558,ash shamiyah,IQ,31.9636,44.5990,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_2

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)